In [14]:
# import module
import pandas as pd
import numpy as np
import re

In [15]:
# load data into dataframe
etf = pd.read_excel('/Users/linyijing/Downloads/Socially Responsible ETFs.xlsx')

In [16]:
# replace missing value to np.nan
etf = etf.replace('--',np.nan)

In [17]:
# print and see first twenty funds
etf.head(20)

,Ticker,Fund Name,Issuer,AUM,Expense Ratio,3-Mo TR,Segment
0,ESGU,iShares ESG Aware MSCI USA ETF,Blackrock,$23.43B,0.0015,0.0348,Equity: U.S. - Total Market
1,ESGD,iShares ESG Aware MSCI EAFE ETF,Blackrock,$7.11B,0.0020,0.0165,Equity: Developed Markets Ex-North America - T...
2,ESGE,iShares ESG Aware MSCI EM ETF,Blackrock,$7.00B,0.0025,0.0147,Equity: Emerging Markets - Total Market
3,ICLN,iShares Global Clean Energy ETF,Blackrock,$6.46B,0.0042,0.0612,Equity: Global Renewable Energy
4,ESGV,Vanguard ESG U.S. Stock ETF,Vanguard,$5.72B,0.0012,0.0355,Equity: U.S. - Total Market
5,XSOE,WisdomTree Emerging Markets ex-State-Owned Ent...,WisdomTree,$4.28B,0.0032,-0.0013,Equity: Emerging Markets - Total Market
6,SUSL,iShares ESG MSCI USA Leaders ETF,Blackrock,$4.18B,0.0010,0.0527,Equity: U.S. - Total Market
7,SUSA,iShares MSCI USA ESG Select ETF,Blackrock,$4.06B,0.0025,0.0439,Equity: U.S. - Total Market
8,USSG,Xtrackers MSCI U.S.A. ESG Leaders Equity ETF,Deutsche Bank,$4.00B,0.0010,0.0498,Equity: U.S. - Total Market
9,DSI,iShares MSCI KLD 400 Social ETF,Blackrock,$3.75B,0.0025,0.0540,Equity: U.S. - Total Market


#### When viewing the dataset, we find that for the data in column "AUM", there are different units (million and billion) for each value, and we want to turn them into same unit so that we can perform analysis on this column. And further change the column name to make views better understand what the value means.


In [18]:
# replace $ with space to make data easier to better perform following processing and analysis
etf['AUM']=etf['AUM'].str.replace('$','')

In [19]:
# convert billion value and million value in column 'AUM' to thousand dollars

etf.AUM = etf.AUM.replace(r'[BM]+$', '', regex=True).astype(float) * \
                 etf.AUM.str.extract(r'[\d\.]+([BM]+)', expand=False)\
                .fillna(1).replace(['B','M'], [10**6, 10**3]).astype(int)


# rename column to better understand the data
etf = etf.rename(columns = {'AUM':'AUM (thousands)'})
etf.head()

,Ticker,Fund Name,Issuer,AUM (thousands),Expense Ratio,3-Mo TR,Segment
0,ESGU,iShares ESG Aware MSCI USA ETF,Blackrock,23430000.0,0.0015,0.0348,Equity: U.S. - Total Market
1,ESGD,iShares ESG Aware MSCI EAFE ETF,Blackrock,7110000.0,0.0020,0.0165,Equity: Developed Markets Ex-North America - T...
2,ESGE,iShares ESG Aware MSCI EM ETF,Blackrock,7000000.0,0.0025,0.0147,Equity: Emerging Markets - Total Market
3,ICLN,iShares Global Clean Energy ETF,Blackrock,6460000.0,0.0042,0.0612,Equity: Global Renewable Energy
4,ESGV,Vanguard ESG U.S. Stock ETF,Vanguard,5720000.0,0.0012,0.0355,Equity: U.S. - Total Market


As we looked at the dataset, for the last column, there are more than Segment information in each value. Therefore, we need to split the column into three columns, seperately represent each information itself, making us to analyze the columns with better overall understanding of the dataset.

In [20]:
# first seperate asset class information from the segment column

etf[['Asset Class','Market']] = etf['Segment'].str.split(':\s+', expand=True)

In [21]:
# then seperate market information from the remaining value in currently market column

etf[['Market','Segment']] = etf['Market'].str.split('-\s+', expand=True)

As we look into the dataset, there are multiple columns that make views have difficulties to understand the relationship and make comparisons to different category in the data. Therefore, it's best to index the data in multiple levels, and we decided to index the data first by the issuer and then by the ticker.

In [22]:
# set two index to Issuer and Ticker and sort these two index

etf1 = etf.set_index(['Issuer','Ticker']).sort_index(level=['Issuer','Ticker'])

In [23]:
etf1.head(50)

Fund Name  \
Issuer                       Ticker                                                      
Allianz                      EMNT         PIMCO Enhanced Short Maturity Active ESG ETF   
                             RAFE                              PIMCO RAFI ESG U.S. ETF   
Ameriprise Financial         ESGN    Columbia Sustainable International Equity Inco...   
                             ESGS          Columbia Sustainable U.S. Equity Income ETF   
Barclays                     GRN                             iPath Series B Carbon ETN   
                             RODI                    Barclays Return on Disability ETN   
                             WIL                      Barclays Women in Leadership ETN   
Beyond Investing             VEGN                               U.S. Vegan Climate ETF   
Blackrock                    BGRN                        iShares Global Green Bond ETF   
                             CRBN              iShares MSCI ACWI Low Carbon Target ETF   
                             DMXF                   iShares ESG Advanced MSCI EAFE ETF   
                             DSI                       iShares MSCI KLD 400 Social ETF   
                             EAGG            iShares ESG Aware U.S. Aggregate Bond ETF   
                             EAOA          iShares ESG Aware Aggressive Allocation ETF   
                             EAOK        iShares ESG Aware Conservative Allocation ETF   
                             EAOM            iShares ESG Aware Moderate Allocation ETF   
                             EAOR              iShares ESG Aware Growth Allocation ETF   
                             EMXF                     iShares ESG Advanced MSCI EM ETF   
                             ESGD                      iShares ESG Aware MSCI EAFE ETF   
                             ESGE                        iShares ESG Aware MSCI EM ETF   
                             ESGU                       iShares ESG Aware MSCI USA ETF   
                             ESML             iShares ESG Aware MSCI USA Small-Cap ETF   
                             EUSB       iShares ESG Advanced Total USD Bond Market ETF   
                             HYXF    iShares ESG Advanced High Yield Corporate Bond...   
                             ICLN                      iShares Global Clean Energy ETF   
                             LDEM                      iShares ESG MSCI EM Leaders ETF   
                             SDG                        iShares MSCI Global Impact ETF   
                             SUSA                      iShares MSCI USA ESG Select ETF   
                             SUSB    iShares ESG Aware 1-5 Year USD Corporate Bond ETF   
                             SUSC             iShares ESG Aware USD Corporate Bond ETF   
                             SUSL                     iShares ESG MSCI USA Leaders ETF   
                             USXF                    iShares ESG Advanced MSCI USA ETF   
                             XJH                  iShares ESG Screened S&P Mid-Cap ETF   
                             XJR                iShares ESG Screened S&P Small-Cap ETF   
                             XVV                      iShares ESG Screened S&P 500 ETF   
CICC                         KCCA          KraneShares California Carbon Allowance ETF   
                             KESG         KraneShares MSCI China ESG Leaders Index ETF   
                             KEUA            KraneShares European Carbon Allowance ETF   
                             KGRN    KraneShares MSCI China Clean Technology Index ETF   
                             KRBN                        KraneShares Global Carbon ETF   
Change Finance               CHGX    Change Finance U.S. Large Cap Fossil Fuel Free...   
Compagnie Lombard Odier SCmA ESUS        ETRACS 2x Leveraged MSCI USA ESG Focus TR ETN   
Crestview                    MDCP                    VictoryShares THB Mid Cap ESG ETF   
                           

With a quick view on the above results dataframe, most of the funds are investing in U.S., which make sense becasuse U.S. is the most well-developed country with soaring economy and all kinds of industries generating tons of revenue each year.

#### To get a clearer view on which type of social issue is for each fund, the funds needed to be categorized into corresonding social issues based on the discription of the fund name.

In [24]:
# create dictionary to map certain word in fund name into categories
grouping_map = {'ESG': 'ESG', 'Energy': 'Energy', 'Carbon': 'Energy', 'Climate': 'Climate',
       'Clean Tech': 'Energy', 'Fossil Fuel': 'Energy','Health':'Health','clean':'Energy',
               'Disability':'Disability','Carbon':'Energy','other':'Other'}

# create object to define what texts should be searched and contribute to the corresponding category
conditions = list(map(etf['Fund Name'].str.contains,grouping_map))

# first set the index to mapping column
etf.set_index('Fund Name')


# create a column to store the initial processing of the category
etf['category'] = np.select(conditions, grouping_map, 'other')


In [25]:
# create a column to store the final version of the category for each fund
etf['Category'] = etf['category'].map(grouping_map)

# delete the column which is no longer useful in the project
del etf['category']

In [26]:
etf['Category'].value_counts()

ESG           87
Other         49
Energy        26
Climate        2
Health         1
Disability     1
Name: Category, dtype: int64

We can see that most of th funds are in ESG category, it may be that because ECG contain the area of environment, socail and governance,the probability for funds and stocks to be relevent to one of these three categories is higher than other categories.

#### In order to quickly fugure out what analysis we should perform to understand ETF market, we should know which issuer is the most influencial investing bank in ETFs markets, in other words, which issuer issued the largest number of ETF portfolio to investers.

In [27]:
# count how many funds have each issuer issed and sort them in descending order to get a better view
etf.groupby('Issuer')['Fund Name'].count().sort_values(ascending=False)

Issuer
Blackrock                                   27
Nuveen Securities                           12
Deutsche Bank                               10
Mirae Asset Global Investments Co., Ltd.     9
Invesco                                      9
State Street Global Advisors                 8
Inspire                                      8
New York Life                                7
Northern Trust                               6
WisdomTree                                   6
CICC                                         5
Toroso Investments                           5
Franklin Templeton                           4
Crestview                                    3
First Trust                                  3
Barclays                                     3
Impact Shares                                3
VanEck                                       3
Vanguard                                     3
Timothy Plan                                 2
Allianz                                      2
Amerip

As we see above analysis that Blackrock has issued 27 funds, which is twice more than the second issuer. We should further analyze on Blackrock, because these 27 funds can provide us with enough information on how the stocks are being piled up as a fund and get to know what casues Blockrock to be the top rank of the fund issuer on number of funds issued. Further, we can see if the revenue Blackrock generate is way much more than those only issued one fund and determine whether it is necessary to issed such a large number of funds in order to make more money.

#### To perform a more objective way of analysis to dertermine which issuer worths the most attention to be analyzed, besides compare issers with the number of funds issued, we can also compare them by the the total asset under management, which can be a better attribute for us to dertermine which issuer is the most influential company in ETF market.

In [28]:
# get the total asset under management for each issuer and sort in descending order
etf.groupby('Issuer')['AUM (thousands)'].sum().sort_values(ascending=False)

Issuer
Blackrock                                   65469720.0
Vanguard                                     8783040.0
Invesco                                      6268550.0
WisdomTree                                   5465830.0
Deutsche Bank                                5417780.0
Nuveen Securities                            4588680.0
First Trust                                  3222990.0
Mirae Asset Global Investments Co., Ltd.     1600000.0
CICC                                         1296330.0
Inspire                                      1128980.0
SS&C                                         1010000.0
New York Life                                 738130.0
Northern Trust                                481340.0
Franklin Templeton                            432510.0
Goldman Sachs                                 349680.0
Allianz                                       199330.0
ETFMG                                         191420.0
VanEck                                        178400.0
Tor

Surpringly or not surprisingly, the issuer generating the highest revenue is Blackrock, which is also the issuer issued the most many funds. Thus, we should look deeper into Blackrock and see what makes it that kind of ability to own huge amount of asset. However, if compare this result to the previous analysis on the number of funds issued by each issuer, we should instead study Vanguard further, because Vanguard only issued two funds and ranked second in asset under management, which worth more time to study further.

#### Further, since we have compared the asset under management and number of funds issued for each issuer, it's time to analyze on assets under management on each fund, to see which fund appeals the most investers. Also, we can take a look at which issuer owns the biggest fund with the largest assets under management. 

In [29]:
# sort the assets under management for each fund 
etf.groupby('Fund Name')['AUM (thousands)'].sum().sort_values(ascending=False)

Fund Name
iShares ESG Aware MSCI USA ETF                              23430000.0
iShares ESG Aware MSCI EAFE ETF                              7110000.0
iShares ESG Aware MSCI EM ETF                                7000000.0
iShares Global Clean Energy ETF                              6460000.0
Vanguard ESG U.S. Stock ETF                                  5720000.0
                                                               ...    
ETRACS 2x Leveraged MSCI USA ESG Focus TR ETN                      0.0
SPDR MSCI ACWI Low Carbon Target ETF                               0.0
SPDR MSCI Emerging Markets Fossil Fuel Reserves Free ETF           0.0
VictoryShares Top Veteran Employers ETF                            0.0
SPDR Bloomberg SASB Corporate Bond ESG Select ETF                  0.0
Name: AUM (thousands), Length: 166, dtype: float64

As we can see from the above analysis, the fund "iShares ESG Aware MSCI USA ETF" owns really large amount of asset under management compared to the others. Further, the amount of asset under management almost four times as the amount for the rank 2 ETF's, which is a huge gap, thus iShares ESG Aware MSCI USA ETF definately need to be studied further into what protfolio and what stocks are within this fund that is capable of having such huge anount of asset under management.

#### Though the asset of management is a method to get to know the quality of the ETF, the hidden expenses also need to be considered as we are investers looking for the maximum return for our investments to the ETF. Thus, analyze the mean of ETF's operating expense for each issuer is crucial for us to understand the expense ratio and consider which ETF should we invest in.

In [30]:
# get the mean percent of Expense Ratio for each of the issuer to see which issuer have the highest and lowest exepense ratio overall
etf.groupby('Issuer')['Expense Ratio'].mean().sort_values(ascending=False)

Issuer
Tuttle Tactical Management, LLC             0.014300
Direxion                                    0.010700
Compagnie Lombard Odier SCmA                0.009500
New Age Alpha Advisors LLC                  0.009000
Red Gate Advisers LLC                       0.008500
Guinness Atkinson Asset Management          0.007900
CICC                                        0.007500
ProcureAM                                   0.007500
Point Bridge Capital                        0.007200
First Trust                                 0.006900
Impact Shares                               0.006633
Inspire                                     0.006562
Toroso Investments                          0.006260
Beyond Investing                            0.006000
Virtus Investment Partners                  0.005900
ProShares                                   0.005800
TrueMark Investments                        0.005800
Timothy Plan                                0.005700
Franklin Templeton                     

As we can see from the above analysis, Humankind USA LLC has the lowest expense ratio and Tuttle Tactical Management, LLC has the highest expense ratio. But what caught my eye is Vanguard, which owns the second high asset under management has the second lowest expense ratio. It may be the reason that because of the hugh amount of asset under management and huge number of investers to split the operating expense, thus the operating expense could be lower in percentage compared to those with small amount of asset under management. Thus,we can still study further on the issuer with the highest expense ration, which is Tuttle Tactical Management, LLC, get to know what services it provide to investers so that have such high expense ratio. But I would consider to further study on Vnaguard to get a close look at how they operate the fund to get the low ratio of operating expense with such large asset under management.

#### In order to see the relationship for the columns 'Segment', 'Asset Class', 'Market' to idenetify if there is a pattern for ETF to have the majority type on the specific categorie for each these three columns. So we can realize what scope of majority of the ETFs cover and if there is a relationship between Asset Class nad Market or between Segment and Asset Class, Which could give us an insight on how ETFs work and come up with a matrix to chose the most profitable ETF by utilizing these three variables for ETFs.

#### I used three combination of these target columns to perfrom frequency distribution analysis to see if there are some interesting insights can draw from it and determine which type of fund may be pursued. 

In [31]:
# extract columns of interest
segment_assetclass_market = etf.iloc[:,-4:-1]

# Multi-column frequency count
count = segment_assetclass_market.groupby(['Segment','Asset Class']).size()
print(count)

Segment                                   Asset Class     
Broad Market, Broad-based                 Fixed Income         3
Broad Market, Broad-based Investment Gr…  Fixed Income         3
Broad Market, Broad-based Investment …    Fixed Income         2
Corporate, Broad-based High Yield         Fixed Income         3
Corporate, Broad-based Hig…               Fixed Income         1
Corporate, Broad-based Investment Grade   Fixed Income         6
Corporate, Broad-based Investment Grade…  Fixed Income         2
Extended Market                           Equity               1
Government, Non-Native Curr…              Fixed Income         1
High Dividend Yield                       Equity               1
Large Cap                                 Equity              30
Large Cap Growth                          Equity               3
Large Cap Value                           Equity               2
Mid Cap                                   Equity               4
Mid Cap Growth                 

As we see from the above frquency of distribution, there are lots number of the funds with segment of total market have an asset class of equity, while the second large number of funds with a large cap segement have an asset class of equity as well. Therefore, we can conclude that most of the funds have an asset class of equity. The reason may be that equity is the most common way and more easily to acquire large amount of money to sustain the operations of the funds. Thus, most of the funds have an asset class of equity.

In [32]:
# Multi-column frequency count
count2 = segment_assetclass_market.groupby(['Market','Asset Class']).size()
print(count2)

Market                               Asset Class     
China                                Equity               3
Developed Markets                    Equity               2
                                     Fixed Income         1
Developed Markets Ex-North America   Equity               6
Developed Markets Ex-U.S.            Equity               4
Developed Markets Water              Equity               1
Emerging Markets                     Equity               9
                                     Fixed Income         1
Emerging Markets Low Carbon          Equity               1
Energy Carbon Credits                Commodities          4
Global                               Equity               6
                                     Fixed Income         4
Global Consumer                      Equity               1
Global Environment                   Equity               6
Global Ex-U.S.                       Equity               2
Global Low Carbon                    Equity   

As we see from the above frquency of distribution, U.S. market have the most number of funds with an asset class of equity. It may be casued by that U.S. is an proper country with well-developed economy, therefore, there are lots of funds to not only support the economy and also make profit from it. Not to mention that among each market, every country have the most funds with the asset class of equity, showing that eqiuty asset class is the most efficient way to accumulate large base amount of asset needed to manage the funds.

In [33]:
# Multi-column frequency count
count3 = segment_assetclass_market.groupby(['Market','Segment']).size()
print(count3)

Market                               Segment                                 
China                                Total Market                                 3
Developed Markets                    Corporate, Broad-based Hig…                  1
                                     Total Market                                 2
Developed Markets Ex-North America   Total Market                                 6
Developed Markets Ex-U.S.            Large Cap                                    3
                                     Total Market                                 1
Emerging Markets                     Government, Non-Native Curr…                 1
                                     Total Market                                 9
Global                               Broad Market, Broad-based                    2
                                     Broad Market, Broad-based Investment …       2
                                     Large Cap                                    

From the above analysis showed that most of the funds are in U.S. market and in the segment of Large Cap. Therefore, we can conclude that funds in U.S. market is making the largest anount of money with Large Cap segment as well as with total market. <br>

Concluded based on the above three analysis of frequency distributions, I would recommend to invest in U.S. market with the equity asset class, as to the segement, it can be both in large cap or total market, depeding on what's the customers' willingness to take the risks. For customers whose investment strategies are risk aversion, I would recommend to invest in funds with total market segment, because it would not be seriously affected if the financial crisis stroke and vice versa. As to both risk averters and risk lovers, I would recommend them choose to invest in U.S. market with equity asset class to have a stable funding ot cash flows to sustain the operating of the funds as well as remain the high quality of monetoring the timely fluctuations in the stock market and quickly make adaption to the funds.

#### We can now provide suggenstions on which fund to invest based on the previous methods. However, just based on one analysis to decide on the best fund is a little bit subjective. Therefore, we want to do another analysis with different variables to have an overall view of the funds and better determine what the company should pursue. The method in the following analysis is to determine the mean 3-month total return percentages factoring in a combination of asset class, market, and segment together.

In [34]:
# get the mean of 3-month total return for the combination of segment, asset class and market
# sort the data in descending order and only get the first five rows to get useful insight 
etf.groupby(['Segment', 'Asset Class', 'Market'])['3-Mo TR'].mean().sort_values(ascending=False).head()

Segment           Asset Class  Market        
Total Market      Equity       India             0.1014
Mid Cap Value     Equity       U.S.              0.0617
Total Market      Equity       North America     0.0585
Large Cap Growth  Equity       U.S.              0.0543
Mid Cap           Equity       U.S.              0.0506
Name: 3-Mo TR, dtype: float64

As we can see from the above results that the top 1 combination which have a 10.14% for 3-month return rate, which is almost half larger than the other combinations in the top5 series showing above. We can see that the top 1 fund is in India market, indicating that India's economy is growing a lot faster than United States. Though funds in U.S. market still have an excellent performance on 3-month total return as a whole, indicating that U.S. still have the edge with its strength and influence on world economics. <br>
Thus I would recommend to go for the fund with the highest 3-month return for now, because it's the fastest way to accumulate your assets and time is money. Further, if the clients are not sure about whether to keep investing tons of money in india market, which is a reletively unstable economic entity compared to the U.S., then it is also a good choice to invest parts of the asset to U.S. market in Mid Cap Value Segment. Last but not least, we noticed that fund with mid cap value segment has a higher 3-month return than with total market, and the reason is that for mid cap value companies, there is still lots potential to grow big faster with lots of visions and hopes within the investers than those of high cap value companies. <br>
To conclude, first invest in India market and later invest in U.S. market with med cap value segment to not only earn money in a quick way but also can sleep well at night for not putting too much money in a market which is not that stable as the market in U.S.

#### Some people might think that holding socially responsible investments is not profitable compared to holding investments that do not address socially responsible causes. For the reason that portfolio with less constraints on choosing the designated category of companies would have the total control over choosing the most profitable or promissing companies that may help generate revenue for the funds. Therefore, the following analysis would suggest whether this kind of claim is correct or not.<br>I found a piece of review on overall stock performance over the time sapn from 2011-2020 and compared this dataset with this project's dataset to validate if the calim that holding socially responsible investments is not profitable compared to holding investments that do not address socially responsible causes.

In [35]:
# load stock market's average annualized returns over the past 10, 30, and 50 years,
# using the S&P 500 as our proxy for the market:

stock = pd.read_html('https://www.fool.com/investing/how-to-invest/stocks/average-stock-market-return/')

# display the data in dataframe
stock[0]

,Period,Annualized Return (Nominal),Annualized Real Return (Adjusted for Inflation),$1 Becomes... (Nominal),$1 Becomes... (Adjusted for Inflation)
0,10 years (2011-2020),13.9%,11.96%,$3.67,$3.10
1,30 years (1991-2020),10.7%,8.3%,$21.25,$10.93
2,50 years (1971-2020),10.9%,6.8%,$177.33,$27.12


As we can see from the dataframe that the annualized return on S&P 500 is around 12% which is quite similar with the highest rank socially responsible ETFs in 3-month total return,10.14%. However, the rest of the performance of socially responsible ETFs are way below 12%. Therefore, the claim maybe true for that socially responsible investments may have more chance to have lower profit compared to those investments which are not socially responsible.